In [1]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import multiprocessing
import logging
import torch
import pandas as pd
from torch_geometric.data import Data, Batch
from torch_geometric.transforms import Distance
import torch_geometric.nn as gnn

from utils import *
import envs

import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from deep_rl import *

from deep_rl.component.envs import DummyVecEnv, make_env

import envs

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)    
    

/home/tgog/.conda/envs/my-rdkit-env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


_ColormakerRegistry()

1
3
5
7
2
4
6
8
9
11
13
15
17
glug glug
reset called
[118.12815655611193, 63.310666530774434, 173.2125075981846, 48.72733628320435, 168.46036720948098, 104.40792978250536, -178.33034393068147, -177.28834591074718]
Data(edge_attr=[168, 9], edge_index=[2, 168], pos=[46, 3], x=[46, 3])


In [2]:
# from pointnet import SAModule, GlobalSAModule, MLP
# from torch_geometric.nn import knn_interpolate
# from torch_geometric.utils import intersection_and_union as i_and_u

# class FPModule(torch.nn.Module):
#     def __init__(self, k, nn):
#         super(FPModule, self).__init__()
#         self.k = k
#         self.nn = nn

#     def forward(self, x, pos, batch, x_skip, pos_skip, batch_skip):
#         x = knn_interpolate(x, pos, pos_skip, batch, batch_skip, k=self.k)
#         if x_skip is not None:
#             x = torch.cat([x, x_skip], dim=1)
#         x = self.nn(x)
#         return x, pos_skip, batch_skip


# class ActorPointNet(torch.nn.Module):
#     def __init__(self, num_classes):
#         super(Net, self).__init__()
#         self.sa1_module = SAModule(0.2, 0.2, MLP([3 + 3, 64, 64, 128]))
#         self.sa2_module = SAModule(0.25, 0.4, MLP([128 + 3, 128, 128, 256]))
#         self.sa3_module = GlobalSAModule(MLP([256 + 3, 256, 512, 1024]))

#         self.fp3_module = FPModule(1, MLP([1024 + 256, 256, 256]))
#         self.fp2_module = FPModule(3, MLP([256 + 128, 256, 128]))
#         self.fp1_module = FPModule(3, MLP([128, 128, 128, 128]))

#         self.lin1 = torch.nn.Linear(128, 128)
#         self.lin2 = torch.nn.Linear(128, 128)
#         self.lin3 = torch.nn.Linear(128, num_classes)

#     def forward(self, data):
#         sa0_out = (data.x, data.pos, data.batch)
#         sa1_out = self.sa1_module(*sa0_out)
#         sa2_out = self.sa2_module(*sa1_out)
#         sa3_out = self.sa3_module(*sa2_out)

#         fp3_out = self.fp3_module(*sa3_out, *sa2_out)
#         fp2_out = self.fp2_module(*fp3_out, *sa1_out)
#         x, _, _ = self.fp1_module(*fp2_out, *sa0_out)

#         x = F.relu(self.lin1(x))
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.lin2(x)
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.lin3(x)
#         return F.log_softmax(x, dim=-1)
    
# class CriticPointNet(torch.nn.Module):
#     def __init__(self):
#         super(CriticPointNet, self).__init__()

#         self.sa1_module = SAModule(0.5, 0.2, MLP([3, 64, 64, 128]))
#         self.sa2_module = SAModule(0.25, 0.4, MLP([128 + 3, 128, 128, 256]))
#         self.sa3_module = GlobalSAModule(MLP([256 + 3, 256, 512, 1024]))

#         self.lin1 = Lin(1024, 512)
#         self.lin2 = Lin(512, 256)
#         self.lin3 = Lin(256, 1)

#     def forward(self, data):
#         sa0_out = (data.x, data.pos, data.batch)
#         sa1_out = self.sa1_module(*sa0_out)
#         sa2_out = self.sa2_module(*sa1_out)
#         sa3_out = self.sa3_module(*sa2_out)
#         x, pos, batch = sa3_out

#         x = F.relu(self.lin1(x))
#         x = F.relu(self.lin2(x))
#         x = self.lin3(x)
#         return x
    
# class PointTorsionNet(torch.nn.Module):
#         def __init__(self):
#             super(PointTorsionNet, self).__init__()

#             self.actor = ActorPointNet(6)
#             self.critic = CriticPointNet()
#         def forward(self, obs, states=None):
#             if states:
#                 hp, cp, hv, cv = states
#                 policy_states = (hp, cp)
#                 value_states = (hv, cv)
#             else:
#                 policy_states = None
#                 value_states = None

#             logits, (hp, cp) = self.actor(obs, policy_states)
#             v, (hv, cv) = self.critic(obs, value_states)

#             dist = torch.distributions.Categorical(logits=logits)
#             action = dist.sample().cuda()
#             log_prob = dist.log_prob(action).unsqueeze(0).cuda()
#             entropy = dist.entropy().unsqueeze(0).cuda()

#             prediction = {
#                 'a': action,
#                 'log_pi_a': log_prob,
#                 'ent': entropy,
#                 'v': v,
#             }

#             return prediction, (hp, cp, hv, cv)

In [3]:
class CriticBatchNet(torch.nn.Module):
    def __init__(self, action_dim, dim, edge_dim):
        super(CriticBatchNet, self).__init__()
        num_features = 3
        self.lin0 = torch.nn.Linear(num_features, dim)
        func_ag = nn.Sequential(nn.Linear(edge_dim, dim), nn.ReLU(), nn.Linear(dim, dim * dim))
        self.conv = gnn.NNConv(dim, dim, func_ag, aggr='mean')
        self.gru = nn.GRU(dim, dim)

        self.set2set = gnn.Set2Set(dim, processing_steps=6)
        self.lin1 = torch.nn.Linear(dim, dim)
        self.lin3 = torch.nn.Linear(dim, 1)

        self.action_dim = action_dim
        self.dim = dim

        self.memory = nn.LSTM(2*dim, dim)

    def forward(self, obs, states=None):
        data, nonring, nrbidx, torsion_list_sizes = obs
        data.to(torch.device(0))

        if states:
            hx, cx = states
        else:
            hx = Variable(torch.zeros(1, data.num_graphs, self.dim)).cuda()
            cx = Variable(torch.zeros(1, data.num_graphs, self.dim)).cuda()

        out = F.relu(self.lin0(data.x.cuda()))
        h = out.unsqueeze(0)

        for i in range(6):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        pool = self.set2set(out, data.batch)
        lstm_out, (hx, cx) = self.memory(pool.view(1,data.num_graphs,-1), (hx, cx))
        out = F.relu(self.lin1(lstm_out))
        v = self.lin3(out)

        return v, (hx, cx)

class ActorBatchNet(torch.nn.Module):
    def __init__(self, action_dim, dim, edge_dim):
        super(ActorBatchNet, self).__init__()
        num_features = 3
        self.lin0 = torch.nn.Linear(num_features, dim)
        func_ag = nn.Sequential(nn.Linear(edge_dim, dim), nn.ReLU(), nn.Linear(dim, dim * dim))
        self.conv = gnn.NNConv(dim, dim, func_ag, aggr='mean')
        self.gru = nn.GRU(dim, dim)

        self.set2set = gnn.Set2Set(dim, processing_steps=6)
        self.lin1 = torch.nn.Linear(5 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, action_dim)

        self.memory = nn.LSTM(2*dim, dim)

        self.action_dim = action_dim
        self.dim = dim

    def forward(self, obs, states=None):
        data, nonring, nrbidx, torsion_list_sizes = obs
        data.to(torch.device(0))

        if states:
            hx, cx = states
        else:
            hx = Variable(torch.zeros(1, data.num_graphs, self.dim)).cuda()
            cx = Variable(torch.zeros(1, data.num_graphs, self.dim)).cuda()

        out = F.relu(self.lin0(data.x.cuda()))
        h = out.unsqueeze(0)

        for i in range(6):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
        pool = self.set2set(out, data.batch)
        lstm_out, (hx, cx) = self.memory(pool.view(1,data.num_graphs,-1), (hx, cx))

        print(lstm_out.shape)
        lstm_out = torch.index_select(
            lstm_out,
            dim=1,
            index=nrbidx
        )
        print(lstm_out.shape)
        
        out = torch.index_select(
            out,
            dim=0,
            index=nonring.view(-1)
        ).view(4, -1, self.dim)
        print(out.shape)

        out = torch.cat([lstm_out,out],0)   #5, num_torsions, self.dim
        out = out.permute(2,1,0).reshape(-1, 5*self.dim) #num_torsions, 5*self.dim
        out = F.relu(self.lin1(out))
        out = self.lin2(out)

        logit = out.split(torsion_list_sizes)
        logit = torch.nn.utils.rnn.pad_sequence(logit).permute(1,0,2)

        return logit, (hx, cx)

class RTGNBatch(torch.nn.Module):
    def __init__(self, action_dim, dim, edge_dim=7, point_dim=3):
        super(RTGNBatch, self).__init__()
        num_features = point_dim
        self.action_dim = action_dim
        self.dim = dim

        self.actor = ActorBatchNet(action_dim, dim, edge_dim=edge_dim)
        self.critic = CriticBatchNet(action_dim, dim, edge_dim=edge_dim)

    def forward(self, obs, states=None, action=None):
        data_list = []
        nr_list = []
        for b, nr in obs:
            data_list += b.to_data_list()
            nr_list.append(torch.LongTensor(nr).cuda())

        b = Batch.from_data_list(data_list)
        so_far = 0
        torsion_batch_idx = []
        torsion_list_sizes = []

        for i in range(b.num_graphs):
            trues = (b.batch == i).view(1, -1)
            nr_list[i] += so_far
            so_far += int((b.batch == i).sum())
            torsion_batch_idx.extend([i]*int(nr_list[i].shape[0]))
            torsion_list_sizes += [nr_list[i].shape[0]]

        nrs = torch.cat(nr_list)
        torsion_batch_idx = torch.LongTensor(torsion_batch_idx).cuda()
        obs = (b, nrs, torsion_batch_idx, torsion_list_sizes)

        if states:
            hp, cp, hv, cv = states
            policy_states = (hp, cp)
            value_states = (hv, cv)
        else:
            policy_states = None
            value_states = None

        logits, (hp, cp) = self.actor(obs, policy_states)
        v, (hv, cv) = self.critic(obs, value_states)

        dist = torch.distributions.Categorical(logits=logits)
        if action is None:
            action = dist.sample()
        action = dist.sample().cuda()
        log_prob = dist.log_prob(action).unsqueeze(0).cuda()
        entropy = dist.entropy().unsqueeze(0).cuda()

        prediction = {
            'a': action,
            'log_pi_a': log_prob,
            'ent': entropy,
            'v': v,
        }

        return prediction, (hp, cp, hv, cv)

In [4]:
#message passing
from torch.autograd import Variable
from torch import nn
from torch_geometric.nn import GATConv
import torch.nn.functional as F



class CriticTransformer(torch.nn.Module):
    def __init__(self, action_dim, dim):
        super(CriticTransformer, self).__init__()
        num_features = 3
        self.lin0 = torch.nn.Linear(num_features, dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

        self.set2set = gnn.Set2Set(dim, processing_steps=6)
        self.lin1 = torch.nn.Linear(dim, dim)
        self.lin3 = torch.nn.Linear(dim, 1)

        self.action_dim = action_dim
        self.dim = dim

        self.memory = nn.LSTM(2*dim, dim)

    def forward(self, obs, states=None):
        data, nonring, nrbidx, torsion_list_sizes = obs

        if states:
            hx, cx = states
        else:
            hx = Variable(torch.zeros(1, data.shape[0], self.dim)).cuda()
            cx = Variable(torch.zeros(1, data.shape[0], self.dim)).cuda()

        out = F.relu(self.lin0(data))
        out = self.transformer_encoder(out)
        
        num_atoms = out.shape[1]
        batch_len = out.shape[0]
        bidxs = []
        
        for i in range(batch_len):
            bidxs.append(torch.ones([num_atoms], dtype=torch.int32) * i)
            
        bidxs = torch.cat(bidxs).long().cuda()
        pool_in = torch.flatten(out, start_dim=0, end_dim=1)  

    
        pool = self.set2set(pool_in, bidxs)
        lstm_out, (hx, cx) = self.memory(pool.view(1,data.shape[0],-1), (hx, cx))
        
        out = F.relu(self.lin1(lstm_out))
        v = self.lin3(out)

        return v, (hx, cx)

class ActorTransformer(torch.nn.Module):
    def __init__(self, action_dim, dim):
        super(ActorTransformer, self).__init__()
        num_features = 3

        self.lin0 = torch.nn.Linear(num_features, dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

        self.set2set = gnn.Set2Set(dim, processing_steps=6)
        self.lin1 = torch.nn.Linear(5 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, action_dim)

        self.memory = nn.LSTM(2*dim, dim)

        self.action_dim = action_dim
        self.dim = dim


    def forward(self, obs, states=None):
        data, nonring, nrbidx, torsion_list_sizes = obs

        
        if states:
            hx, cx = states
        else:
            hx = Variable(torch.zeros(1, data.shape[0], self.dim)).cuda()
            cx = Variable(torch.zeros(1, data.shape[0], self.dim)).cuda()            
            
        out = F.relu(self.lin0(data))
        out = self.transformer_encoder(out)        
        
        num_atoms = out.shape[1]
        batch_len = out.shape[0]
        bidxs = []
        
        for i in range(batch_len):
            bidxs.append(torch.ones([num_atoms], dtype=torch.int32) * i)
            
        bidxs = torch.cat(bidxs).long().cuda()
        pool_in = torch.flatten(out, start_dim=0, end_dim=1)  

    
        pool = self.set2set(pool_in, bidxs)
        lstm_out, (hx, cx) = self.memory(pool.view(1,data.shape[0],-1), (hx, cx))

        lstm_out = torch.index_select(
            lstm_out,
            dim=1,
            index=nrbidx
        )  
        
        out = out.view(1, num_atoms * batch_len, self.dim)
        out = torch.index_select(
            out,
            dim=1,
            index=nonring.view(-1)
        )
    
        out = out.view(4, -1, self.dim)       

        out = torch.cat([lstm_out,out],0)   #5, num_torsions, self.dim
        out = out.permute(2,1,0).reshape(-1, 5*self.dim) #num_torsions, 5*self.dim
        out = F.relu(self.lin1(out))
        out = self.lin2(out)

        logit = out.split(torsion_list_sizes)
        logit = torch.nn.utils.rnn.pad_sequence(logit).permute(1,0,2)
        return logit, (hx, cx)
    
class GraphTransformerBatch(torch.nn.Module):
    def __init__(self, action_dim, dim, point_dim=3):
        super(GraphTransformerBatch, self).__init__()
        num_features = point_dim
        self.action_dim = action_dim
        self.dim = dim

        self.actor = ActorTransformer(action_dim, dim)
        self.critic = CriticTransformer(action_dim, dim)

    def forward(self, obs, states=None, action=None):
        # data_list = []
        # nr_list = []
        # for b, nr in obs:
        #     data_list.append(b.x.cuda())
        #     nr_list.append(torch.LongTensor(nr).cuda())

        # torsion_batch_idx = []
        # torsion_list_sizes = []

        # for i in range(len(obs)):
        #     trues = (b.batch == i).view(1, -1)
        #     nr_list[i] += so_far
        #     so_far += int((b.batch == i).sum())
        #     torsion_batch_idx.extend([i]*int(nr_list[i].shape[0]))
        #     torsion_list_sizes += [nr_list[i].shape[0]]

        # nrs = torch.cat(nr_list)
        # torsion_batch_idx = torch.LongTensor(torsion_batch_idx).cuda()
        # obs = (b, nrs, torsion_batch_idx, torsion_list_sizes)

        data_list = []
        nr_list = []
        for b, nr in obs:
            data_list.append(b.x.cuda())
            nr_list.append(torch.LongTensor(nr).cuda())

        b = torch.nn.utils.rnn.pad_sequence(data_list, batch_first=True)
        nrs = torch.nn.utils.rnn.pad_sequence(nr_list, batch_first=True)
        
#         print(b.shape) #torch.Size([4, 15, 3])
        max_atoms = b.shape[1]
    
#         print(nrs.shape) #4,10,4 batch first

        torsion_batch_idx = []
        torsion_list_sizes = []
        
        for i in range(len(obs)):
            nrs[i] += max_atoms * i
            torsion_batch_idx.extend([i]*int(nr_list[i].shape[0]))
            torsion_list_sizes += [nr_list[i].shape[0]]
            
        torsion_batch_idx = torch.LongTensor(torsion_batch_idx).cuda()
        obs = (b, nrs, torsion_batch_idx, torsion_list_sizes)
            
        if states:
            hp, cp, hv, cv = states
            policy_states = (hp, cp)
            value_states = (hv, cv)
        else:
            policy_states = None
            value_states = None

        logits, (hp, cp) = self.actor(obs, states=policy_states)
 
        v, (hv, cv) = self.critic(obs, states=value_states)

        dist = torch.distributions.Categorical(logits=logits)
        if action is None:
            action = dist.sample()
        action = dist.sample().cuda()
        log_prob = dist.log_prob(action).unsqueeze(0).cuda()
        entropy = dist.entropy().unsqueeze(0).cuda()

        prediction = {
            'a': action,
            'log_pi_a': log_prob,
            'ent': entropy,
            'v': v,
        }

        return prediction, (hp, cp, hv, cv)


In [5]:
import envs
m = GraphTransformerBatch(6, 128)
m.to(torch.device('cuda'))




GraphTransformerBatch(
  (actor): ActorTransformer(
    (lin0): Linear(in_features=3, out_features=128, bias=True)
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=128, out_features=128, bias=True)
          )
         

In [16]:
task = AdaTask('AllTenTorsionSetDense-v0', seed=random.randint(0,7e4), num_envs=1, single_process=True)
x = task.reset()

prediction, recurrent_states = m(x)
# next_states, rewards, terminals, info = task.step(to_np(prediction['a']))


seed is  55125
step time mean nan
reset called





[-179.93808835469966, -179.59844696587746, -175.1584517632214, 87.52936455389786, -59.330298348286064, -57.44536318176025, 163.35002294919693, -138.95710912180365, 170.29548351706333, -141.55473571861535]
torch.Size([1, 1, 128])
torch.Size([10])
torch.Size([1, 10, 128])
torch.Size([1, 18, 128])
torch.Size([1, 10, 4])
torch.Size([1, 40, 128])
torch.Size([4, 10, 128])


In [7]:
Variable(torch.zeros(1, 1, 128)).cuda()

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       device='cuda:0')

In [8]:
# from models import RTGNBatch

# model = RTGNBatch(6, 128)
# model.to(torch.device('cuda'))

In [9]:
def a2c_feature(**kwargs):
    generate_tag(kwargs)
    kwargs.setdefault('log_level', 0)
    config = Config()
    config.merge(kwargs)

    config.num_workers = 1
    single_process = (config.num_workers == 1)
    config.task_fn = lambda: AdaTask('OneSet-v0',num_envs=config.num_workers, seed=random.randint(0,1e5), single_process=single_process)
    
    lr = 7e-5 * np.sqrt(config.num_workers)
    
    config.optimizer_fn = lambda params: torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-5) #learning_rate #alpha #epsilon
    config.network = model
    config.discount = 0.9999 # gamma
    config.use_gae = False
    config.gae_tau = 0.95
    config.value_loss_weight = 0.25 # vf_coef
    config.entropy_weight = 0.0005 #ent_coef
    config.rollout_length = 5 # n_steps
    config.gradient_clip = 0.5 #max_grad_norm
    config.max_steps = 5000000
    config.save_interval = 10000
    config.eval_interval = 2000
    config.eval_episodes = 2
    config.eval_env = AdaTask('Diff-v0', seed=random.randint(0,7e4))
    config.state_normalizer = DummyNormalizer()
    
    agent = A2CRecurrentEvalAgent(config)
    return agent

In [10]:
# mkdir('log')
# mkdir('tf_log')
# set_one_thread()
# select_device(0)
# tag='rtgn_node_memory_new_HP'
# agent = a2c_feature(tag=tag)


In [11]:
# run_steps(agent)
#     

In [12]:
# import glob
# glob.glob('data/*diff*')

In [13]:
# from importlib import reload
# reload(graphenvironments)


In [14]:
# model = RTGN(6, 64)
# model.load_state_dict(torch.load('data/A2CRecurrentAgent-gnn_new_model_basic_carbon-120000.model'))
# model.to(torch.device('cuda'))

In [15]:
# m = Chem.MolFromMolFile('lignin_guaiacyl.mol')
# m = Chem.AddHs(m)
# AllChem.EmbedMultipleConfs(m, numConfs=1, numThreads=0)
# res = AllChem.MMFFOptimizeMoleculeConfs(m, numThreads=0)

# mol = m

# nonring, ring = TorsionFingerprints.CalculateTorsionLists(mol)